# DESCRIPTION

This code is used to convert images into numpy arrays so that they can be utilized in time series analysis.


## input



*   Folder with images we want to process.


## output

*   .npy file with images in a sequence, we can specify the size of the sequence.

*   .npy file with annotations, so we can determine whether the plant is frosty or not.

* .npy file with image names.

imports

In [ ]:

import pandas as pd

import numpy as np

import os

from matplotlib import image as imagelib

import glob

from PIL import Image

import matplotlib.pyplot as plt

drive_path = ""
kit_ml_dir = "/KIT ML/Fenotypizace - vzchazeni"
if os.path.isdir("/Volumes/GoogleDrive/Shared drives"):
    drive_path = "/Volumes/GoogleDrive/Shared drives"
elif os.path.isdir("G:/Shared drives"):
    drive_path = "G:/Shared drives"
elif not os.path.isdir(drive_path):
    from google.colab import drive
    drive.mount('/content/gdrive')
    drive_path = "/content/gdrive/Shareddrives"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.9 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
# opraveno, Honza
save_path= drive_path+"/KIT ML 2/Data/numpy/cropped64/"

#directory for input files
# data_dir = kit_ml_dir + "/Model/Data, na kterých jsem zkoušel rozpoznání klíčení/"
feno_dir = drive_path + "/Fenotypizace/EA2021-01Ohnoutkova/"
data_path = kit_ml_dir + "/Data"
# teoreticky vyměnit jen tuto složku
data_images_path = drive_path + "/KIT ML 2" + "/Data/cropped64/"

annotation_file = "EA2021_01.xlsx"
#[10,11,13,14,15,17,40,41,44,45,46,48]
#12,
trays_train = [10,11,12,13,14,15,16,18,41,47]
trays_val = [17,40,42,44,45]
trays_test = [43,46,48]

#length of the sliding window
n_time_steps = [3,4,5]
n_data_range = [3,4,5,7,9]
#step, we need to identify precisely the time when it emerges
slide_step = 1

# v kódu vypustit toto
img_crop = 160

#model hyper parameters
val_split = 0.2
num_epochs = 5

#RGB
colors = 3

#img_size
img_size = 64

# FUNCTIONS

#### load annotations

#### load files

In [ ]:
def load_annotations(annotation_path, use_trays):
    emergence = pd.read_excel(annotation_path, sheet_name = "Results - emergence", usecols = "B,C,D,G,H")
    df = emergence.loc[emergence['tray_location'].isin(use_trays)]
    return df

def load_annotations(annotation_path, use_trays):
    emergence = pd.read_excel(annotation_path, sheet_name = "Results - emergence", usecols = "B,C,D,G,H")
    df = emergence.loc[emergence['tray_location'].isin(use_trays)]
    return df

def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

def load_from_dir(df, data_directory, crop_size,length):
    labels = np.array([])
    imgs_array = np.array([])
    previous_img = None

    for index1, item in df.iterrows():
        well_id = str(item["well_id"])
        well_id = well_id.replace("_","-")
        directory = data_directory  + "Tray_" + str(int(item["tray_location"])) + "/" + well_id
        #print(directory)
        index = 0
        germ_class = 0

        # I do not need all files, instead take emerged +- length files
        files = sorted(glob.glob(directory + "/*.png"))
        #print(item["time_of_first_occurence"])
        if not isinstance(item["time_of_first_occurence"], str) or item["time_of_first_occurence"] == "-":
            continue
        emerged_index = index_containing_substring(files, item["time_of_first_occurence"].split("_")[1])
        ## toto je klíčové
        files_ranged = files[emerged_index-length:emerged_index+length] # tohel nám určuje rozsah
        #print(emerged_index)
        #print(len(files_ranged))

        df_occurrence = item["time_of_first_occurence"].split("_")[1] + ".png"

        for filename in files_ranged:
            filename_occurrence = filename.split("_")[2] + ".png"
            if df_occurrence == filename_occurrence:
                germ_class = 1

            # we need images of the same size
            np_image = np.array(Image.open(filename))
            #np_image = imagelib.imread(filename)
            # print(np_image.shape)
            # print(root)

            # if the image is smaller - probably corrupted
            if np_image.shape[0] < img_size or np_image.shape[1] < img_size:
                np_image = previous_img
                print("Image error:"+filename)
            else:
                previous_img = np_image

            #print(np_image.shape)

            #add image to output array
            if len(imgs_array) == 0:
                imgs_array = np.array([np_image])
            else:
                imgs_array = np.append(imgs_array, [np_image], axis=0)

            #output labels array
            if len(labels) == 0:
                labels = np.array([germ_class])
            else:
                labels = np.append(labels, [germ_class], axis=0)

        index += 1

    print(imgs_array.shape)
    print(labels.shape)
    return imgs_array, labels

from matplotlib import image as imagelib
import glob
from PIL import Image

def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

def load_from_dir_test(df, data_directory, crop_size,length):
    labels = np.array([])
    imgs_array = np.array([])
    image_names = []
    previous_img = None

    for index1, item in df.iterrows():
        well_id = str(item["well_id"])
        well_id = well_id.replace("_","-")
        directory = data_directory  + "Tray_" + str(int(item["tray_location"])) + "/" + well_id
        index = 0
        germ_class = 0

        # I do not need all files, instead take emerged +- length files
        files = sorted(glob.glob(directory + "/*.png"))
        #print(item["time_of_first_occurence"])
        if not isinstance(item["time_of_first_occurence"], str) or item["time_of_first_occurence"] == "-":
            continue
        emerged_index = index_containing_substring(files, item["time_of_first_occurence"].split("_")[1])
        ## toto je klíčové
        files_ranged = files #PRY TO MA BYT ZAKOMENTOAVNE, JA TO UPRAVIL, ŽE TO VEZME VŠECHNY OBRÁZKY VE SLOŽCE, TAKŽE LENGTH ANRGUMENT JE IRELEVANTNÍ
        #print(emerged_index)
        #print(len(files_ranged))

        df_occurrence = item["time_of_first_occurence"].split("_")[1] + ".png"

        for filename in files_ranged:
            filename_occurrence = filename.split("_")[2] + ".png"
            if df_occurrence == filename_occurrence:
                germ_class = 1

            # we need images of the same size
            np_image = np.array(Image.open(filename))
            #np_image = imagelib.imread(filename)
            # print(np_image.shape)
            # print(root)

            # if the image is smaller - probably corrupted
            if np_image.shape[0] < img_size or np_image.shape[1] < img_size:
                np_image = previous_img
                print("Image error:"+filename)
            else:
                previous_img = np_image

            #print(np_image.shape)

            #add image to output array
            if len(imgs_array) == 0:
                imgs_array = np.array([np_image])
            else:
                imgs_array = np.append(imgs_array, [np_image], axis=0)

            #output labels array
            if len(labels) == 0:
                labels = np.array([germ_class])
            else:
                labels = np.append(labels, [germ_class], axis=0)

            image_names.append(os.path.basename(filename))

        index += 1

    print(imgs_array.shape)
    print(labels.shape)
    return imgs_array, labels, image_names


def generate_samples_test(images, labels, image_names, step, time_steps):
    img_trays = np.array([images[0:time_steps]])
    annotations_array = np.array([0])
    image_names_trays = np.array([image_names[0:time_steps]])

    for i in range(1, images.shape[0] - time_steps, step):
        # if all items in a sliding window are equal, we use them for training
        # arr = labels[i:i + time_steps]
        # if np.all(arr == arr[0]):
        img_trays = np.append(img_trays, [images[i:i + time_steps]], 0)
        annotations_array = np.append(annotations_array, [labels[i]], 0)
        image_names_trays = np.append(image_names_trays, [image_names[i:i + time_steps]], 0)

    print(img_trays.shape)
    return img_trays, annotations_array, image_names_trays

def generate_samples(images, labels, step, time_steps):
    img_trays = np.array([images[0:time_steps]])
    annotations_array = np.array([0])

    for i in range(1,  images.shape[0]- time_steps, step):
        #if all items in a sliding window are equal than we use them for training
        arr= labels[i:i + time_steps]
        if np.all(arr == arr[0]):
            img_trays = np.append(img_trays,[images[i:i+time_steps]],0)
            annotations_array = np.append(annotations_array, [labels[i]], 0)

    print(img_trays.shape)
    return img_trays, annotations_array


# load data from file system and save it as numpy file
def load_data_images(images_path,annotations_path, use_trays, dataset ="train",sizes = 9, time_steps = 3, img_crop_size = 160):
    #load images from filesystem
    annotations_pd = load_annotations(annotations_path, use_trays)

    #np.save(images_path +"/" + dataset + "_raw_data_"+str(size)+"_"+str(use_trays), raw_images)
    #np.save(images_path +"/" + dataset + "_raw_annotations_"+str(size)+"_"+str(use_trays), raw_annotations)

    #generate samples
    for time_step in time_steps:
        print("step size: "+str(time_step))
        # size není potřeba
        for size in sizes:
            if size<time_step: continue
            print ("range size: "+str(size))
            path = images_path +"/npy_nonormalize/data_"+str(time_step)+"_"+str(use_trays) + ".npy"
            if os.path.isfile(path):
                print("exists")
                continue

            raw_images, raw_annotations = load_from_dir(annotations_pd, images_path, img_crop_size, size)

            (samples, labels) = generate_samples(raw_images, raw_annotations,slide_step, time_step)

            np.save(save_path + "data_"+ str(img_size) + "_" + str(time_step) + "_" + str(size) + "_" + str(use_trays), samples)
            np.save(save_path + "annotations_"+ str(img_size) + "_" + str(time_step) + "_" + str(size) + "_" + str(use_trays), labels)

    return

# load data from file system and save it as numpy file
def load_data_images_test(images_path,annotations_path, use_trays, dataset ="train",sizes = 9, time_steps = 3, img_crop_size = 160):
    # load images from filesystem
    annotations_pd = load_annotations(annotations_path, use_trays)

    # generate samples
    for time_step in time_steps:
        print("step size: " + str(time_step))
        for size in sizes:
            if size < time_step:
                continue
            print("range size: " + str(size))
            path = images_path + "/npy_nonormalize/data_" + str(time_step) + "_" + str(use_trays) + ".npy"
            if os.path.isfile(path):
                print("exists")
                continue

            raw_images, raw_annotations, image_names = load_from_dir_test(annotations_pd, images_path, img_crop_size, size)

            (samples, labels, image_names_trays) = generate_samples_test(raw_images, raw_annotations, image_names, slide_step, time_step)

            np.save(save_path + "data_" + str(img_size) + "_" + str(time_step) + "_" + str(size) + "_" + str(use_trays), samples)
            np.save(save_path + "annotations_" + str(img_size) + "_" + str(time_step) + "_" + str(size) + "_" + str(use_trays), labels)
            np.save(save_path + "image_names_" + str(img_size) + "_" + str(time_step) + "_" + str(size) + "_" + str(use_trays), image_names_trays)

    return



### Generate the samples - reorganize, take always batch of X images

**It is necessary to change the paths where it will be saved, and we can also change the file name.**


### Input parameters

Here are the parameters we are using. In our case, we have data stored in multiple main folders and their subfolders, and our code has been adjusted accordingly.

If you want to preprocess different data, you'll need to adjust the parameters significantly and likely modify the code as well.

### load data

# EXECUTION

In [ ]:
n_time_steps = [3]  # tady potřebujem pouze test set
n_data_range = [9]
#load_data_images(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_train,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)
#load_data_images(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_val,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)
load_data_images_test(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_test,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)


n_time_steps = [4]
n_data_range = [10]
load_data_images(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_train,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)
load_data_images(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_val,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)
load_data_images_test(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_test,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)

n_time_steps = [2]
n_data_range = [8]
load_data_images(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_train,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)
load_data_images(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_val,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)
load_data_images_test(data_images_path,feno_dir+"/"+annotation_file,use_trays=trays_test,sizes=n_data_range, time_steps=n_time_steps, img_crop_size=img_crop)